# NLTK Complete Guide - Section 14: Text Classification

## What is Text Classification?

**Text Classification** (also called Text Categorization) is the task of automatically assigning predefined categories or labels to text documents. It's one of the most fundamental and widely-used NLP tasks.

### Real-World Applications

| Application | Description | Example |
|-------------|-------------|---------|
| **Spam Detection** | Classify emails as spam or not spam | Gmail's spam filter |
| **Sentiment Analysis** | Determine if text is positive/negative | Product review ratings |
| **Topic Classification** | Assign topics to articles | News categorization |
| **Language Detection** | Identify the language of text | Google Translate detection |
| **Intent Recognition** | Understand user intent in chatbots | Customer service bots |
| **Content Moderation** | Detect inappropriate content | Social media filtering |

### The Classification Pipeline

```
┌─────────────┐    ┌──────────────┐    ┌─────────────┐    ┌────────────┐
│   Raw Text  │ →  │   Feature    │ →  │   Machine   │ →  │ Predicted  │
│  Documents  │    │  Extraction  │    │  Learning   │    │   Label    │
└─────────────┘    └──────────────┘    └─────────────┘    └────────────┘
```

### What This Notebook Covers

| Topic | Description |
|-------|-------------|
| **Feature Extraction** | Converting text to numerical features |
| **Naive Bayes Classifier** | A probabilistic classification algorithm |
| **Training & Evaluation** | How to train and measure performance |
| **Cross-Validation** | Robust evaluation technique |
| **Practical Examples** | Gender classification, movie review sentiment |

In [1]:
import nltk
import random
from collections import Counter

nltk.download('movie_reviews', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('names', quiet=True)

from nltk.corpus import movie_reviews, names, stopwords
from nltk.classify import NaiveBayesClassifier, accuracy
from nltk.classify.util import apply_features
from nltk.tokenize import word_tokenize

### Understanding the Imports

| Import | Purpose |
|--------|---------|
| `movie_reviews` | Corpus of 2000 movie reviews labeled positive/negative |
| `names` | Corpus of male and female first names |
| `stopwords` | Common words to filter out (the, is, at, etc.) |
| `NaiveBayesClassifier` | The classifier algorithm we'll use |
| `accuracy` | Function to calculate classification accuracy |
| `apply_features` | Memory-efficient feature extraction |

## 14.1 Introduction to Text Classification

### The Core Concept

Text classification works by:
1. **Converting text to features** (numbers that represent the text)
2. **Learning patterns** from labeled examples
3. **Predicting labels** for new, unseen text

### Types of Classification

| Type | Classes | Example |
|------|---------|---------|
| **Binary** | 2 classes | Spam vs Not Spam |
| **Multi-class** | 3+ classes (one per doc) | News topics (sports, politics, tech) |
| **Multi-label** | Multiple labels per doc | Movie genres (action AND comedy) |

### Naive Bayes Classifier

We'll use **Naive Bayes**, a probabilistic classifier based on Bayes' theorem:

$$P(class|features) = \frac{P(features|class) \cdot P(class)}{P(features)}$$

**Why "Naive"?** It assumes all features are independent of each other (which is rarely true in practice, but the algorithm still works remarkably well!)

### Why Naive Bayes for Text?

✅ **Fast to train** - Linear time complexity  
✅ **Works well with high-dimensional data** - Text has many features  
✅ **Requires little training data** - Good for small datasets  
✅ **Handles irrelevant features** - Robust to noise

## 14.2 Simple Example: Gender Classification from Names

Let's start with a simple example: predicting gender from first names. This is a great introduction because:
- The data is simple (just names)
- The features are intuitive (letters in the name)
- The results are interpretable

### The Key Concept: Feature Extraction

**Features** are the measurable properties we extract from data. For names, we might use:
- Last letter (names ending in 'a' are often female)
- First letter
- Length of name
- Last two letters

The classifier will learn which features are most predictive of each class.

In [2]:
def gender_features(name):
    """
    Extract features from a name for gender classification.
    
    Features are stored in a dictionary where:
    - Keys are feature names (strings)
    - Values are the feature values (can be any type)
    
    The classifier will learn which features correlate with which gender.
    
    Args:
        name: A person's first name (string)
    
    Returns:
        Dictionary of features
    """
    return {
        'last_letter': name[-1].lower(),      # Most predictive feature!
        'last_two': name[-2:].lower(),        # "ia", "na" → often female
        'first_letter': name[0].lower(),      # Less predictive
        'length': len(name),                   # Names length varies by gender
    }

# Test the feature extractor with example names
print("FEATURE EXTRACTION EXAMPLES")
print("=" * 55)
print("\nExtracting features from different names:\n")

test_names = ['John', 'Mary', 'Alexandra', 'Michael', 'Sarah']
for name in test_names:
    features = gender_features(name)
    print(f"  {name:<12} → {features}")

print("\n💡 Notice: Female names often end in vowels (a, y, h)")
print("   Male names often end in consonants (n, l, d)")

FEATURE EXTRACTION EXAMPLES

Extracting features from different names:

  John         → {'last_letter': 'n', 'last_two': 'hn', 'first_letter': 'j', 'length': 4}
  Mary         → {'last_letter': 'y', 'last_two': 'ry', 'first_letter': 'm', 'length': 4}
  Alexandra    → {'last_letter': 'a', 'last_two': 'ra', 'first_letter': 'a', 'length': 9}
  Michael      → {'last_letter': 'l', 'last_two': 'el', 'first_letter': 'm', 'length': 7}
  Sarah        → {'last_letter': 'h', 'last_two': 'ah', 'first_letter': 's', 'length': 5}

💡 Notice: Female names often end in vowels (a, y, h)
   Male names often end in consonants (n, l, d)


In [3]:
# STEP 1: Prepare the labeled training data
# Each item is a tuple: (name, gender_label)

male_names = [(name, 'male') for name in names.words('male.txt')]
female_names = [(name, 'female') for name in names.words('female.txt')]

# Combine all names
all_names = male_names + female_names

# IMPORTANT: Shuffle the data to avoid ordering bias
# (All male names first, then all female would bias training)
random.seed(42)  # For reproducibility
random.shuffle(all_names)

print("TRAINING DATA PREPARED")
print("=" * 45)
print(f"\n📊 Dataset Statistics:")
print(f"   Total names:  {len(all_names):,}")
print(f"   Male names:   {len(male_names):,}")
print(f"   Female names: {len(female_names):,}")

print(f"\n📝 Sample data (first 5 after shuffling):")
for name, gender in all_names[:5]:
    print(f"   {name:<15} → {gender}")

TRAINING DATA PREPARED

📊 Dataset Statistics:
   Total names:  7,944
   Male names:   2,943
   Female names: 5,001

📝 Sample data (first 5 after shuffling):
   Raye            → female
   Marita          → female
   Fey             → female
   Vittoria        → female
   Dannie          → female


In [4]:
# STEP 2: Create feature sets and split into train/test
# Feature set = (features_dict, label) for each example

featuresets = [(gender_features(name), gender) for (name, gender) in all_names]

# Split into training (80%) and test (20%) sets
# Training set: Used to train the classifier
# Test set: Used to evaluate performance on unseen data
train_size = int(len(featuresets) * 0.8)
train_set = featuresets[:train_size]
test_set = featuresets[train_size:]

print("TRAIN/TEST SPLIT")
print("=" * 45)
print(f"""
┌─────────────────────────────────────────────┐
│  Training set:  {len(train_set):>5} examples (80%)     │
│  Test set:      {len(test_set):>5} examples (20%)     │
└─────────────────────────────────────────────┘

Why split the data?
• Training set: Classifier learns patterns from this
• Test set: Measures how well it generalizes to NEW data

⚠️ NEVER test on training data - it leads to over-optimistic results!
""")

TRAIN/TEST SPLIT

┌─────────────────────────────────────────────┐
│  Training set:   6355 examples (80%)     │
│  Test set:       1589 examples (20%)     │
└─────────────────────────────────────────────┘

Why split the data?
• Training set: Classifier learns patterns from this
• Test set: Measures how well it generalizes to NEW data

⚠️ NEVER test on training data - it leads to over-optimistic results!



In [5]:
# STEP 3: Train the Naive Bayes classifier
# This is where the magic happens - the algorithm learns from the training data

print("TRAINING CLASSIFIER...")
print("=" * 45)

classifier = NaiveBayesClassifier.train(train_set)

print("✅ Training complete!\n")

# STEP 4: Evaluate on test set
# accuracy() returns the proportion of correct predictions
acc = accuracy(classifier, test_set)

print("EVALUATION RESULTS")
print("=" * 45)
print(f"""
┌─────────────────────────────────────────────┐
│           ACCURACY: {acc:.1%}               │
└─────────────────────────────────────────────┘

This means the classifier correctly predicts gender
for {acc*100:.1f}% of names it has never seen before!

For reference:
• 50% = Random guessing (coin flip)
• 70-80% = Good for this task
• 100% = Perfect (impossible for this task)
""")

TRAINING CLASSIFIER...
✅ Training complete!

EVALUATION RESULTS

┌─────────────────────────────────────────────┐
│           ACCURACY: 78.4%               │
└─────────────────────────────────────────────┘

This means the classifier correctly predicts gender
for 78.4% of names it has never seen before!

For reference:
• 50% = Random guessing (coin flip)
• 70-80% = Good for this task
• 100% = Perfect (impossible for this task)



In [6]:
# STEP 5: Inspect most informative features
# This tells us WHAT the classifier learned

print("MOST INFORMATIVE FEATURES")
print("=" * 60)
print("""
This shows which features are most useful for distinguishing
male vs female names. The ratio indicates how much more likely
a feature value is for one gender vs the other.

Example: "last_letter = 'a'" with ratio 35.3:1 means names
ending in 'a' are 35.3x more likely to be female than male.
""")
print("-" * 60)
classifier.show_most_informative_features(10)
print("-" * 60)
print("\n💡 The last letter is clearly the most informative feature!")

MOST INFORMATIVE FEATURES

This shows which features are most useful for distinguishing
male vs female names. The ratio indicates how much more likely
a feature value is for one gender vs the other.

Example: "last_letter = 'a'" with ratio 35.3:1 means names
ending in 'a' are 35.3x more likely to be female than male.

------------------------------------------------------------
Most Informative Features
                last_two = 'la'           female : male   =     64.0 : 1.0
                last_two = 'ra'           female : male   =     50.4 : 1.0
             last_letter = 'a'            female : male   =     41.3 : 1.0
                last_two = 'us'             male : female =     38.5 : 1.0
             last_letter = 'k'              male : female =     37.2 : 1.0
                last_two = 'ia'           female : male   =     35.6 : 1.0
                last_two = 'ta'           female : male   =     27.8 : 1.0
                last_two = 'rd'             male : female =     21.7

In [7]:
# STEP 6: Make predictions on new names
# Let's test with names not in the training data

test_names = ['Michael', 'Jessica', 'Alex', 'Taylor', 'Jordan', 'Emily', 'James', 'Pat']

print("PREDICTIONS ON NEW NAMES")
print("=" * 55)
print(f"{'Name':<12} {'Prediction':<12} {'Confidence':<12} {'Notes'}")
print("-" * 55)

for name in test_names:
    # Extract features
    features = gender_features(name)
    
    # Get prediction
    prediction = classifier.classify(features)
    
    # Get probability distribution
    prob = classifier.prob_classify(features)
    confidence = prob.prob(prediction)
    
    # Add notes for ambiguous names
    notes = ""
    if confidence < 0.7:
        notes = "⚠️ Uncertain"
    elif name in ['Alex', 'Taylor', 'Jordan', 'Pat']:
        notes = "(Gender-neutral)"
    
    print(f"{name:<12} {prediction:<12} {confidence:>8.1%}     {notes}")

print("\n💡 Note: Some names (Alex, Taylor, Jordan) are gender-neutral,")
print("   so low confidence is expected and appropriate!")

PREDICTIONS ON NEW NAMES
Name         Prediction   Confidence   Notes
-------------------------------------------------------
Michael      male            55.0%     ⚠️ Uncertain
Jessica      female         100.0%     
Alex         male            81.9%     (Gender-neutral)
Taylor       male            96.1%     (Gender-neutral)
Jordan       male            77.7%     (Gender-neutral)
Emily        female          61.4%     ⚠️ Uncertain
James        male            90.8%     
Pat          male            92.0%     (Gender-neutral)

💡 Note: Some names (Alex, Taylor, Jordan) are gender-neutral,
   so low confidence is expected and appropriate!


## 14.3 Document Classification: Movie Reviews

Now let's tackle a more realistic task: **sentiment classification of movie reviews**.

### The Dataset: NLTK Movie Reviews Corpus

- **2000 movie reviews** from IMDb
- **Labeled**: 1000 positive, 1000 negative
- **Average length**: ~700 words per review

### The Challenge

Unlike names (short, simple), movie reviews are:
- **Long** (hundreds of words)
- **Complex** (varied vocabulary)
- **High-dimensional** (many possible features)

### Feature Engineering for Documents

For documents, we use **bag-of-words** features:
- Each word is a potential feature
- Feature value = whether word is present in document
- Ignore word order (hence "bag")

```
"I love this movie" → {contains(love): True, contains(hate): False, ...}
```

In [8]:
# Load and explore the movie reviews corpus
# Each document is a list of words paired with its category

documents = [
    (list(movie_reviews.words(fileid)), category)
    for category in movie_reviews.categories()
    for fileid in movie_reviews.fileids(category)
]

# Shuffle to mix positive and negative reviews
random.seed(42)
random.shuffle(documents)

print("MOVIE REVIEWS DATASET")
print("=" * 60)
print(f"""
📊 Dataset Statistics:
   Total documents:  {len(documents)}
   Categories:       {movie_reviews.categories()}
   
   Positive reviews: {len([d for d, c in documents if c == 'pos'])}
   Negative reviews: {len([d for d, c in documents if c == 'neg'])}
""")

# Show a sample
sample_doc, sample_label = documents[0]
print(f"📝 Sample Review ({sample_label}):")
print(f"   First 15 words: {sample_doc[:15]}")
print(f"   Total words: {len(sample_doc)}")

# Show what reviews look like
print(f"\n   Raw text preview: '{' '.join(sample_doc[:30])}...'")

MOVIE REVIEWS DATASET

📊 Dataset Statistics:
   Total documents:  2000
   Categories:       ['neg', 'pos']

   Positive reviews: 1000
   Negative reviews: 1000

📝 Sample Review (neg):
   First 15 words: ['mr', '.', 'bean', ',', 'a', 'bumbling', 'security', 'guard', 'from', 'england', 'is', 'sent', 'to', 'la', 'to']
   Total words: 642

   Raw text preview: 'mr . bean , a bumbling security guard from england is sent to la to help with the grandiose homecoming of a masterpiece american painting . the first two words...'


In [9]:
# BUILD THE VOCABULARY (Feature Selection)
# We can't use ALL words as features - too many (overfitting, slow, memory)
# Instead, we select the most common words as our feature vocabulary

# Get all words from the corpus
all_words = [w.lower() for w in movie_reviews.words() if w.isalpha()]
word_freq = Counter(all_words)

# Use the 2000 most common words as features
# Why 2000? Balance between coverage and efficiency
common_words = [w for w, f in word_freq.most_common(2000)]

print("VOCABULARY SELECTION")
print("=" * 50)
print(f"""
📚 Corpus Statistics:
   Total word occurrences: {len(all_words):,}
   Unique words:           {len(word_freq):,}
   
🎯 Feature Selection:
   Using top 2000 words as features
   
📊 Most Common Words:
   {word_freq.most_common(10)}
   
💡 Why limit features?
   • Too many features → overfitting
   • Rare words don't help generalization
   • Faster training and prediction
""")

VOCABULARY SELECTION

📚 Corpus Statistics:
   Total word occurrences: 1,329,753
   Unique words:           38,889

🎯 Feature Selection:
   Using top 2000 words as features

📊 Most Common Words:
   [('the', 76529), ('a', 38106), ('and', 35576), ('of', 34123), ('to', 31937), ('is', 25195), ('in', 21822), ('s', 18513), ('it', 16107), ('that', 15924)]

💡 Why limit features?
   • Too many features → overfitting
   • Rare words don't help generalization
   • Faster training and prediction



In [10]:
def document_features(document, word_features):
    """
    Extract bag-of-words features from a document.
    
    For each word in our vocabulary, create a binary feature
    indicating whether that word appears in the document.
    
    Args:
        document: List of words in the document
        word_features: List of words to use as features
    
    Returns:
        Dictionary of features: {contains(word): True/False}
    """
    # Convert document to set for O(1) lookup
    document_words = set(document)
    
    features = {}
    for word in word_features:
        # Feature name is descriptive: "contains(love)"
        features[f'contains({word})'] = (word in document_words)
    
    return features

# Demonstrate feature extraction
print("DOCUMENT FEATURE EXTRACTION")
print("=" * 55)
print("\nConverting a document to a feature dictionary:")

sample_features = document_features(documents[0][0], common_words[:10])
print("\nSample features (first 10 vocabulary words):")
for feat, value in sample_features.items():
    marker = "✓" if value else "✗"
    print(f"   {marker} {feat}: {value}")

print(f"""
💡 Key Points:
   • Binary features: word present (True) or not (False)
   • Same features for ALL documents (consistent representation)
   • Order doesn't matter (bag-of-words assumption)
""")

DOCUMENT FEATURE EXTRACTION

Converting a document to a feature dictionary:

Sample features (first 10 vocabulary words):
   ✓ contains(the): True
   ✓ contains(a): True
   ✓ contains(and): True
   ✓ contains(of): True
   ✓ contains(to): True
   ✓ contains(is): True
   ✓ contains(in): True
   ✓ contains(s): True
   ✓ contains(it): True
   ✓ contains(that): True

💡 Key Points:
   • Binary features: word present (True) or not (False)
   • Same features for ALL documents (consistent representation)
   • Order doesn't matter (bag-of-words assumption)



In [11]:
# Create feature sets for all documents
# This may take a moment due to the size of the data

print("Creating feature sets for all 2000 documents...")

featuresets = [
    (document_features(doc, common_words), category)
    for (doc, category) in documents
]

print("✅ Feature extraction complete!")

# Split into training and test sets (80/20 split)
train_set = featuresets[:1600]  # 1600 for training
test_set = featuresets[1600:]    # 400 for testing

print(f"""
TRAIN/TEST SPLIT
{'=' * 45}
   Training set: {len(train_set)} documents
   Test set:     {len(test_set)} documents
   
   Each document → {len(common_words)} binary features
   Total feature values: {len(train_set) * len(common_words):,}
""")

Creating feature sets for all 2000 documents...
✅ Feature extraction complete!

TRAIN/TEST SPLIT
   Training set: 1600 documents
   Test set:     400 documents

   Each document → 2000 binary features
   Total feature values: 3,200,000



In [12]:
# Train the classifier
print("TRAINING SENTIMENT CLASSIFIER")
print("=" * 45)
print("Training Naive Bayes on 1600 movie reviews...")
print("(This may take a few seconds)\n")

classifier = NaiveBayesClassifier.train(train_set)

print("✅ Training complete!")

# Evaluate on test set
acc = accuracy(classifier, test_set)

print(f"""
EVALUATION RESULTS
{'=' * 45}

┌─────────────────────────────────────────────┐
│         ACCURACY: {acc:.1%}                  │
└─────────────────────────────────────────────┘

Interpreting the result:
• {acc*100:.1f}% of reviews correctly classified
• Baseline (random): 50%
• Our improvement: {(acc-0.5)*100:.1f} percentage points

This is a solid result for bag-of-words features!
""")

TRAINING SENTIMENT CLASSIFIER
Training Naive Bayes on 1600 movie reviews...
(This may take a few seconds)

✅ Training complete!

EVALUATION RESULTS

┌─────────────────────────────────────────────┐
│         ACCURACY: 81.5%                  │
└─────────────────────────────────────────────┘

Interpreting the result:
• 81.5% of reviews correctly classified
• Baseline (random): 50%
• Our improvement: 31.5 percentage points

This is a solid result for bag-of-words features!



In [13]:
# Analyze what the classifier learned
print("MOST INFORMATIVE FEATURES FOR SENTIMENT")
print("=" * 60)
print("""
These are the words that best distinguish positive from negative reviews.
The ratio shows how much more likely a word is in one class vs the other.
""")
print("-" * 60)
classifier.show_most_informative_features(15)
print("-" * 60)
print("""
💡 Observations:
   • "outstanding", "wonderfully" → Strong positive indicators
   • "worst", "waste", "poorly" → Strong negative indicators
   • The classifier learned meaningful sentiment words!
""")

MOST INFORMATIVE FEATURES FOR SENTIMENT

These are the words that best distinguish positive from negative reviews.
The ratio shows how much more likely a word is in one class vs the other.

------------------------------------------------------------
Most Informative Features
   contains(wonderfully) = True              pos : neg    =     10.3 : 1.0
        contains(seagal) = True              neg : pos    =      9.6 : 1.0
   contains(outstanding) = True              pos : neg    =      8.5 : 1.0
         contains(damon) = True              pos : neg    =      7.0 : 1.0
         contains(mulan) = True              pos : neg    =      6.4 : 1.0
         contains(awful) = True              neg : pos    =      5.8 : 1.0
         contains(flynt) = True              pos : neg    =      5.7 : 1.0
          contains(lame) = True              neg : pos    =      5.6 : 1.0
    contains(ridiculous) = True              neg : pos    =      5.6 : 1.0
        contains(poorly) = True              neg

## 14.4 Improved Feature Extraction

Our basic bag-of-words approach can be improved! Common enhancements:

### Feature Engineering Techniques

| Technique | Description | Benefit |
|-----------|-------------|---------|
| **Remove stopwords** | Filter out common words (the, is, a) | Reduce noise |
| **Add metadata features** | Document length, punctuation counts | Capture style |
| **Bigrams** | Word pairs ("not good") | Capture negation |
| **TF-IDF weighting** | Weight rare words higher | Better discrimination |

### Stopwords

**Stopwords** are common words that usually don't carry sentiment meaning:
- Articles: "the", "a", "an"
- Prepositions: "in", "on", "at"
- Pronouns: "it", "they", "we"

Removing them:
- Reduces feature space
- Focuses on content words
- Often improves accuracy

In [14]:
# Get English stopwords
stop_words = set(stopwords.words('english'))
print(f"Number of stopwords: {len(stop_words)}")
print(f"Examples: {list(stop_words)[:10]}")

def improved_features(document, word_features):
    """
    Extract improved features with stopword filtering and metadata.
    
    Improvements over basic bag-of-words:
    1. Exclude stopwords from word features
    2. Add document-level metadata features
    
    Args:
        document: List of words
        word_features: Vocabulary list
    
    Returns:
        Feature dictionary
    """
    # Normalize words (lowercase, alphabetic only)
    document_words = set(w.lower() for w in document if w.isalpha())
    
    features = {}
    
    # Word presence features (excluding stopwords)
    for word in word_features:
        if word not in stop_words:
            features[f'contains({word})'] = (word in document_words)
    
    # METADATA FEATURES - capture document characteristics
    features['doc_length_long'] = len(document) > 500  # Long review?
    features['has_exclamation'] = '!' in ' '.join(document)  # Excitement?
    features['has_question'] = '?' in ' '.join(document)  # Questioning?
    
    return features

print("\n✅ Improved feature extractor defined!")
print("""
Improvements:
• Stopwords excluded from vocabulary
• Added metadata: length, punctuation
""")

Number of stopwords: 198
Examples: ['because', 'y', 'hers', 'his', 'mightn', 'aren', 'their', 'in', 'o', 's']

✅ Improved feature extractor defined!

Improvements:
• Stopwords excluded from vocabulary
• Added metadata: length, punctuation



In [15]:
# Create filtered vocabulary (no stopwords)
filtered_words = [w for w in common_words if w not in stop_words][:1500]

print("IMPROVED VOCABULARY")
print("=" * 50)
print(f"Original vocabulary: {len(common_words)} words")
print(f"After removing stopwords: {len(filtered_words)} words")
print(f"Reduction: {len(common_words) - len(filtered_words)} words removed")

# Create new feature sets with improved features
print("\nCreating improved feature sets...")
featuresets_improved = [
    (improved_features(doc, filtered_words), category)
    for (doc, category) in documents
]

train_improved = featuresets_improved[:1600]
test_improved = featuresets_improved[1600:]

print("✅ Improved feature sets ready!")

IMPROVED VOCABULARY
Original vocabulary: 2000 words
After removing stopwords: 1500 words
Reduction: 500 words removed

Creating improved feature sets...
✅ Improved feature sets ready!


In [16]:
# Train improved classifier
print("TRAINING IMPROVED CLASSIFIER")
print("=" * 50)

classifier_improved = NaiveBayesClassifier.train(train_improved)
acc_improved = accuracy(classifier_improved, test_improved)

print(f"""
COMPARISON: Original vs Improved
{'=' * 50}

┌──────────────────────────────────────────────────┐
│  Original Accuracy:  {acc:.1%}                     │
│  Improved Accuracy:  {acc_improved:.1%}                     │
│  Change:             {(acc_improved-acc)*100:+.1f} percentage points     │
└──────────────────────────────────────────────────┘

💡 Feature engineering can significantly impact performance!
   Even simple improvements like removing stopwords help.
""")

TRAINING IMPROVED CLASSIFIER

COMPARISON: Original vs Improved

┌──────────────────────────────────────────────────┐
│  Original Accuracy:  81.5%                     │
│  Improved Accuracy:  79.8%                     │
│  Change:             -1.7 percentage points     │
└──────────────────────────────────────────────────┘

💡 Feature engineering can significantly impact performance!
   Even simple improvements like removing stopwords help.



## 14.5 Cross-Validation

### The Problem with Simple Train/Test Split

A single 80/20 split has issues:
- Results depend on which examples end up in test set
- High variance in accuracy estimates
- Might get "lucky" or "unlucky" split

### K-Fold Cross-Validation

**Cross-validation** provides more reliable estimates by:
1. Splitting data into K equal "folds"
2. Using each fold once as test set, rest as training
3. Averaging results across all K experiments

```
Fold 1: [TEST] [Train] [Train] [Train] [Train]
Fold 2: [Train] [TEST] [Train] [Train] [Train]
Fold 3: [Train] [Train] [TEST] [Train] [Train]
Fold 4: [Train] [Train] [Train] [TEST] [Train]
Fold 5: [Train] [Train] [Train] [Train] [TEST]
```

### Benefits

✅ Uses all data for both training and testing  
✅ More stable accuracy estimates  
✅ Better estimate of real-world performance  
✅ Standard practice in machine learning

In [17]:
def cross_validate(featuresets, num_folds=5):
    """
    Perform k-fold cross validation.
    
    Process:
    1. Split data into num_folds equal parts
    2. For each fold:
       - Use that fold as test set
       - Use remaining folds as training set
       - Train classifier and measure accuracy
    3. Return average accuracy across all folds
    
    Args:
        featuresets: List of (features, label) tuples
        num_folds: Number of folds (default 5)
    
    Returns:
        Average accuracy and list of fold accuracies
    """
    fold_size = len(featuresets) // num_folds
    accuracies = []
    
    print(f"Performing {num_folds}-fold cross-validation...")
    print(f"Each fold: {fold_size} examples\n")
    
    for i in range(num_folds):
        # Define test fold boundaries
        test_start = i * fold_size
        test_end = test_start + fold_size
        
        # Split data
        test_fold = featuresets[test_start:test_end]
        train_fold = featuresets[:test_start] + featuresets[test_end:]
        
        # Train and evaluate
        classifier = NaiveBayesClassifier.train(train_fold)
        acc = accuracy(classifier, test_fold)
        accuracies.append(acc)
        
        print(f"Fold {i+1}: {acc:.2%} (trained on {len(train_fold)}, tested on {len(test_fold)})")
    
    avg_accuracy = sum(accuracies) / len(accuracies)
    return avg_accuracy, accuracies

In [18]:
# Run cross-validation on improved features
print("5-FOLD CROSS-VALIDATION RESULTS")
print("=" * 55)

avg_acc, fold_accs = cross_validate(featuresets_improved, num_folds=5)

print(f"""
{'=' * 55}
SUMMARY
{'=' * 55}
   Individual fold accuracies: {[f'{a:.2%}' for a in fold_accs]}
   
   Average Accuracy: {avg_acc:.2%}
   Std Deviation:    {(sum((a-avg_acc)**2 for a in fold_accs)/len(fold_accs))**0.5:.2%}
   
💡 Cross-validation gives us confidence that our {avg_acc:.1%} 
   accuracy is a reliable estimate, not just luck!
""")

5-FOLD CROSS-VALIDATION RESULTS
Performing 5-fold cross-validation...
Each fold: 400 examples

Fold 1: 78.00% (trained on 1600, tested on 400)
Fold 2: 78.25% (trained on 1600, tested on 400)
Fold 3: 80.50% (trained on 1600, tested on 400)
Fold 4: 81.00% (trained on 1600, tested on 400)
Fold 5: 79.75% (trained on 1600, tested on 400)

SUMMARY
   Individual fold accuracies: ['78.00%', '78.25%', '80.50%', '81.00%', '79.75%']

   Average Accuracy: 79.50%
   Std Deviation:    1.19%

💡 Cross-validation gives us confidence that our 79.5% 
   accuracy is a reliable estimate, not just luck!



## 14.6 Confusion Matrix and Evaluation Metrics

### Beyond Accuracy

Accuracy alone can be misleading. Consider:
- 95% accuracy on spam detection sounds great...
- But if 95% of emails are NOT spam, predicting "not spam" always gives 95% accuracy!

### Confusion Matrix

A **confusion matrix** shows the detailed breakdown of predictions:

```
                    Predicted
                 Neg      Pos
Actual  Neg   [  TN   |   FP  ]    TN = True Negative
        Pos   [  FN   |   TP  ]    FP = False Positive
                                   FN = False Negative
                                   TP = True Positive
```

### Key Metrics

| Metric | Formula | What it Measures |
|--------|---------|------------------|
| **Precision** | TP / (TP + FP) | Of predicted positives, how many are correct? |
| **Recall** | TP / (TP + FN) | Of actual positives, how many did we find? |
| **F1 Score** | 2 × (P × R) / (P + R) | Harmonic mean of precision and recall |

### When to Use What?

- **Spam filter**: High precision (don't mark good emails as spam!)
- **Disease diagnosis**: High recall (don't miss sick patients!)
- **Balanced tasks**: F1 score (balance both)

In [19]:
def evaluate_classifier(classifier, test_set):
    """
    Calculate comprehensive evaluation metrics.
    
    Computes:
    - Confusion matrix
    - Precision, Recall, F1 for each class
    
    Args:
        classifier: Trained classifier
        test_set: List of (features, label) tuples
    
    Returns:
        confusion: Confusion matrix as nested dict
        metrics: Precision, recall, F1 for each class
    """
    # Get predictions
    predictions = [classifier.classify(features) for features, label in test_set]
    actual = [label for features, label in test_set]
    
    # Build confusion matrix
    labels = list(set(actual))
    confusion = {true_label: {pred_label: 0 for pred_label in labels} 
                 for true_label in labels}
    
    for true, pred in zip(actual, predictions):
        confusion[true][pred] += 1
    
    # Calculate metrics for each class
    metrics = {}
    for label in labels:
        # True Positives: predicted label correctly
        tp = confusion[label][label]
        
        # False Positives: predicted label but was wrong
        fp = sum(confusion[other][label] for other in labels if other != label)
        
        # False Negatives: should have predicted label but didn't
        fn = sum(confusion[label][other] for other in labels if other != label)
        
        # Calculate metrics (handle division by zero)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics[label] = {
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'support': tp + fn  # Total actual examples of this class
        }
    
    return confusion, metrics

print("✅ Evaluation function defined!")

✅ Evaluation function defined!


In [20]:
# Evaluate classifier with detailed metrics
confusion, metrics = evaluate_classifier(classifier_improved, test_improved)

print("DETAILED EVALUATION RESULTS")
print("=" * 55)

print("""
CONFUSION MATRIX
(Rows = Actual, Columns = Predicted)
""")
print(f"{'':>15} {'Predicted':<25}")
print(f"{'Actual':<15} {'neg':<12} {'pos':<12}")
print("-" * 40)
for true_label in ['neg', 'pos']:
    print(f"{true_label:<15}", end='')
    for pred_label in ['neg', 'pos']:
        count = confusion[true_label][pred_label]
        print(f"{count:<12}", end='')
    print()

print(f"""

INTERPRETING THE MATRIX:
• Diagonal (neg→neg, pos→pos): Correct predictions ✓
• Off-diagonal: Errors ✗
  - neg→pos: Negative reviews predicted as positive (False Positive)
  - pos→neg: Positive reviews predicted as negative (False Negative)
""")

print("\nPER-CLASS METRICS")
print("-" * 55)
print(f"{'Class':<10} {'Precision':<12} {'Recall':<12} {'F1':<10} {'Support'}")
print("-" * 55)
for label in ['neg', 'pos']:
    m = metrics[label]
    print(f"{label:<10} {m['precision']:<12.2%} {m['recall']:<12.2%} {m['f1']:<10.2%} {m['support']}")

# Calculate macro averages
macro_precision = sum(m['precision'] for m in metrics.values()) / len(metrics)
macro_recall = sum(m['recall'] for m in metrics.values()) / len(metrics)
macro_f1 = sum(m['f1'] for m in metrics.values()) / len(metrics)

print("-" * 55)
print(f"{'Macro Avg':<10} {macro_precision:<12.2%} {macro_recall:<12.2%} {macro_f1:<10.2%}")

DETAILED EVALUATION RESULTS

CONFUSION MATRIX
(Rows = Actual, Columns = Predicted)

                Predicted                
Actual          neg          pos         
----------------------------------------
neg            169         29          
pos            52          150         


INTERPRETING THE MATRIX:
• Diagonal (neg→neg, pos→pos): Correct predictions ✓
• Off-diagonal: Errors ✗
  - neg→pos: Negative reviews predicted as positive (False Positive)
  - pos→neg: Positive reviews predicted as negative (False Negative)


PER-CLASS METRICS
-------------------------------------------------------
Class      Precision    Recall       F1         Support
-------------------------------------------------------
neg        76.47%       85.35%       80.67%     198
pos        83.80%       74.26%       78.74%     202
-------------------------------------------------------
Macro Avg  80.13%       79.81%       79.70%    


## 14.7 Complete Text Classifier Class

Let's wrap everything we've learned into a reusable, production-ready class.

### Design Goals

1. **Easy to use** - Simple train/predict interface
2. **Flexible** - Works with raw text or tokenized documents
3. **Configurable** - Adjustable number of features
4. **Informative** - Provides probability estimates and feature analysis

### Class Structure

```python
TextClassifier
├── __init__(num_features)      # Initialize
├── train(documents, labels)    # Train the model
├── predict(document)           # Classify new text
├── predict_proba(document)     # Get probabilities
├── evaluate(documents, labels) # Test accuracy
└── show_features(n)            # Show top features
```

In [21]:
class TextClassifier:
    """
    A complete, reusable text classification pipeline.
    
    This class encapsulates the entire text classification workflow:
    1. Vocabulary building
    2. Feature extraction
    3. Model training
    4. Prediction and evaluation
    
    Example:
        clf = TextClassifier(num_features=1500)
        clf.train(train_docs, train_labels)
        prediction = clf.predict("This movie is great!")
        accuracy = clf.evaluate(test_docs, test_labels)
    """
    
    def __init__(self, num_features=1500):
        """
        Initialize the classifier.
        
        Args:
            num_features: Number of top words to use as features (default 1500)
        """
        self.num_features = num_features
        self.word_features = None  # Will be set during training
        self.classifier = None     # Will be set during training
        self.stop_words = set(stopwords.words('english'))
        print(f"TextClassifier initialized (max {num_features} features)")
    
    def extract_features(self, document):
        """
        Extract features from a single document.
        
        Handles both raw text (string) and tokenized documents (list).
        
        Args:
            document: String or list of tokens
        
        Returns:
            Feature dictionary
        """
        # Handle raw text input
        if isinstance(document, str):
            document = word_tokenize(document.lower())
        
        # Create set of document words for fast lookup
        doc_words = set(w.lower() for w in document if w.isalpha())
        
        # Build feature dictionary
        features = {}
        for word in self.word_features:
            features[f'contains({word})'] = (word in doc_words)
        
        return features
    
    def train(self, documents, labels):
        """
        Train the classifier on labeled documents.
        
        Steps:
        1. Build vocabulary from all documents
        2. Extract features from each document
        3. Train Naive Bayes classifier
        
        Args:
            documents: List of documents (strings or token lists)
            labels: List of labels (same length as documents)
        
        Returns:
            self (for method chaining)
        """
        print(f"Training on {len(documents)} documents...")
        
        # Step 1: Build vocabulary
        all_words = []
        for doc in documents:
            if isinstance(doc, str):
                doc = word_tokenize(doc.lower())
            # Only keep alphabetic words, exclude stopwords
            all_words.extend([w.lower() for w in doc 
                            if w.isalpha() and w.lower() not in self.stop_words])
        
        # Select top N words by frequency
        word_freq = Counter(all_words)
        self.word_features = [w for w, f in word_freq.most_common(self.num_features)]
        print(f"Vocabulary: {len(self.word_features)} words")
        
        # Step 2: Create feature sets
        featuresets = [
            (self.extract_features(doc), label)
            for doc, label in zip(documents, labels)
        ]
        
        # Step 3: Train classifier
        self.classifier = NaiveBayesClassifier.train(featuresets)
        print("✅ Training complete!")
        
        return self
    
    def predict(self, document):
        """
        Predict the class for a document.
        
        Args:
            document: Text to classify (string or token list)
        
        Returns:
            Predicted class label
        """
        features = self.extract_features(document)
        return self.classifier.classify(features)
    
    def predict_proba(self, document):
        """
        Get probability distribution over classes.
        
        Args:
            document: Text to classify
        
        Returns:
            Dictionary {class: probability}
        """
        features = self.extract_features(document)
        prob_dist = self.classifier.prob_classify(features)
        return {label: prob_dist.prob(label) for label in prob_dist.samples()}
    
    def evaluate(self, documents, labels):
        """
        Evaluate accuracy on test data.
        
        Args:
            documents: Test documents
            labels: True labels
        
        Returns:
            Accuracy score (0 to 1)
        """
        featuresets = [
            (self.extract_features(doc), label)
            for doc, label in zip(documents, labels)
        ]
        return accuracy(self.classifier, featuresets)
    
    def show_features(self, n=10):
        """
        Display the most informative features.
        
        Args:
            n: Number of features to show
        """
        print(f"\nTop {n} Most Informative Features:")
        print("-" * 50)
        self.classifier.show_most_informative_features(n)

print("✅ TextClassifier class defined!")

✅ TextClassifier class defined!


In [22]:
# DEMONSTRATION: Using the TextClassifier class

# Prepare data
docs = [list(movie_reviews.words(fid)) for fid in movie_reviews.fileids()]
labels = [movie_reviews.categories(fid)[0] for fid in movie_reviews.fileids()]

# Shuffle
combined = list(zip(docs, labels))
random.shuffle(combined)
docs, labels = zip(*combined)

# Split
train_docs, test_docs = docs[:1600], docs[1600:]
train_labels, test_labels = labels[:1600], labels[1600:]

print("TEXTCLASSIFIER DEMONSTRATION")
print("=" * 55)

# Create and train classifier
clf = TextClassifier(num_features=1500)
clf.train(train_docs, train_labels)

# Evaluate
acc = clf.evaluate(test_docs, test_labels)
print(f"\n📊 Test Accuracy: {acc:.2%}")

# Show what it learned
clf.show_features(10)

TEXTCLASSIFIER DEMONSTRATION
TextClassifier initialized (max 1500 features)
Training on 1600 documents...
Vocabulary: 1500 words
✅ Training complete!

📊 Test Accuracy: 79.75%

Top 10 Most Informative Features:
--------------------------------------------------
Most Informative Features
         contains(mulan) = True              pos : neg    =      7.0 : 1.0
   contains(wonderfully) = True              pos : neg    =      6.6 : 1.0
          contains(lame) = True              neg : pos    =      6.5 : 1.0
    contains(ridiculous) = True              neg : pos    =      5.8 : 1.0
         contains(awful) = True              neg : pos    =      5.6 : 1.0
        contains(wasted) = True              neg : pos    =      5.3 : 1.0
           contains(era) = True              pos : neg    =      5.1 : 1.0
        contains(poorly) = True              neg : pos    =      4.8 : 1.0
         contains(worst) = True              neg : pos    =      4.3 : 1.0
         contains(bland) = True       

In [23]:
# Test with completely new reviews (not from the corpus)
test_reviews = [
    "This movie was absolutely fantastic! Great acting and storyline. A masterpiece!",
    "Terrible film. Complete waste of time and money. Very disappointing experience.",
    "It was okay. Not great, not terrible. Just an average movie overall.",
]

print("\nPREDICTIONS ON NEW REVIEWS")
print("=" * 65)

for i, review in enumerate(test_reviews, 1):
    prediction = clf.predict(review)
    probs = clf.predict_proba(review)
    
    # Determine confidence
    confidence = max(probs.values())
    emoji = "😊" if prediction == 'pos' else "😞"
    
    print(f"\n📝 Review {i}: \"{review[:50]}...\"")
    print(f"   {emoji} Prediction: {prediction.upper()}")
    print(f"   📊 Confidence: pos={probs['pos']:.1%}, neg={probs['neg']:.1%}")
    
    # Interpretation
    if confidence > 0.8:
        certainty = "High confidence"
    elif confidence > 0.6:
        certainty = "Moderate confidence"
    else:
        certainty = "Low confidence (ambiguous)"
    print(f"   💡 {certainty}")

print("\n" + "=" * 65)
print("✅ The classifier successfully generalizes to new, unseen reviews!")


PREDICTIONS ON NEW REVIEWS

📝 Review 1: "This movie was absolutely fantastic! Great acting ..."
   😞 Prediction: NEG
   📊 Confidence: pos=0.0%, neg=100.0%
   💡 High confidence

📝 Review 2: "Terrible film. Complete waste of time and money. V..."
   😞 Prediction: NEG
   📊 Confidence: pos=0.0%, neg=100.0%
   💡 High confidence

📝 Review 3: "It was okay. Not great, not terrible. Just an aver..."
   😞 Prediction: NEG
   📊 Confidence: pos=0.0%, neg=100.0%
   💡 High confidence

✅ The classifier successfully generalizes to new, unseen reviews!


## Summary & Quick Reference

### Text Classification Pipeline

```
┌──────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│  1. Collect  │ →  │  2. Extract  │ →  │  3. Train    │ →  │  4. Predict  │
│  Labeled     │    │  Features    │    │  Classifier  │    │  New Data    │
│  Data        │    │              │    │              │    │              │
└──────────────┘    └──────────────┘    └──────────────┘    └──────────────┘
```

### Key NLTK Functions

| Function | Purpose | Example |
|----------|---------|---------|
| `NaiveBayesClassifier.train()` | Train classifier | `clf = NaiveBayesClassifier.train(train_set)` |
| `classifier.classify()` | Predict single example | `label = clf.classify(features)` |
| `classifier.prob_classify()` | Get probabilities | `probs = clf.prob_classify(features)` |
| `accuracy()` | Evaluate performance | `acc = accuracy(clf, test_set)` |
| `show_most_informative_features()` | Inspect learning | `clf.show_most_informative_features(10)` |

### Feature Set Format

```python
featuresets = [
    ({'feature1': value1, 'feature2': value2}, 'label1'),
    ({'feature1': value3, 'feature2': value4}, 'label2'),
    ...
]
```

### Evaluation Metrics Cheat Sheet

| Metric | Formula | Use When |
|--------|---------|----------|
| **Accuracy** | Correct / Total | Balanced classes |
| **Precision** | TP / (TP + FP) | Cost of false positives high |
| **Recall** | TP / (TP + FN) | Cost of false negatives high |
| **F1** | 2·P·R / (P+R) | Balance P and R |

### Best Practices

1. **Always shuffle data** before splitting
2. **Use cross-validation** for reliable estimates
3. **Start simple**, then add complexity
4. **Analyze errors** to improve features
5. **Remove stopwords** for cleaner features
6. **Don't test on training data!**

### Next Steps

- Explore other classifiers (Decision Trees, SVM, etc.)
- Try different feature engineering techniques
- Build a real application (spam filter, review analyzer)
- Section 15: Corpus Management for handling large datasets